In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
alpha = 0.05
class color:
    BOLD = '\033[1m'
    END = '\033[0m'
    GREEN = '\033[92m'
df = pd.read_excel('Домашнее задание (исходник).xlsx')

In [2]:
# Задача 1
#Постройте группировку, по которой видно, какое количество заказов было для каждой пары животное - порода 
#и какой средний чек был по этой паре

df_gr = df.groupby(['animal', 'species']).agg(cnt_orders = ('id_order', 'count'),
                     avg_check = ('check', 'mean')).reset_index()
df_gr

,animal,species,cnt_orders,avg_check
0,Кошка,Домашняя обыкновенная,1037,26252.648987
1,Кошка,Домашняя породистая,375,27404.896000
2,Кошка,Уличная,598,24168.826087
3,Собака,Домашняя обыкновенная,920,26733.493478
4,Собака,Домашняя породистая,249,20443.963855
5,Собака,Уличная,774,25679.635659
6,Экзотическое животное,Экзотическое животное,893,28748.667413


In [3]:
### Построим необходимые функции для автоматизации
# Проверка только по животному
def checker(a1, a2):

    df1 = df[df['animal']==a1]['check']
    df2 = df[df['animal']==a2]['check']

    df1_avg = df1.mean()
    df2_avg = df2.mean()

    s, p = ttest_ind(df1, df2)

    if p < alpha:
        print(color.GREEN + 'Между выборками есть статистически значимая разница' + color.END)
        if df1_avg > df2_avg:
            print(color.BOLD + 'Средний чек больше по животному:', a1 + color.END)
        elif df2_avg > df1_avg:
            print(color.BOLD + 'Средний чек больше по животному:', a2 + color.END)
        
    elif p > alpha:
        print('Между выборками нет статистически значимой разницы')


# Проверка по животному и виду
def checker_a_s(a_s1, a_s2):

    df1 = df[df['animal']==a_s1[0]][['check', 'species']]
    df1 = df1[df1['species']==a_s1[1]]['check']

    df2 = df[df['animal']==a_s2[0]][['check', 'species']]
    df2 = df2[df2['species']==a_s2[1]]['check']

    df1_avg = df1.mean()
    df2_avg = df2.mean()

    s, p = ttest_ind(df1, df2)

    if p < alpha:
        print(color.GREEN + 'Между выборками есть статистически значимая разница' + color.END)
        if df1_avg > df2_avg:
            print(color.BOLD + 'Средний чек больше по животному:', a_s1[0], a_s1[1] + color.END)
        elif df2_avg > df1_avg:
            print(color.BOLD + 'Средний чек больше по животному:', a_s2[0], a_s2[1] + color.END)
    elif p > alpha:
        print('Между выборками нет статистически значимой разницы')
        

In [4]:
# Задача 2 Ответьте на следующие вопросы, используя функцию ttest_ind:

In [5]:
# 2.1 Какой средний чек выше - по собакам или по кошкам?
a1 = 'Собака'
a2 = 'Кошка'
checker(a1, a2)

Между выборками нет статистически значимой разницы


In [6]:
# 2.2 Какой средний чек выше - по экзотическим животным или по собакам?
a1 = 'Экзотическое животное'
a2 = 'Собака'
checker(a1, a2)

Между выборками есть статистически значимая разница
Средний чек больше по животному: Экзотическое животное


In [7]:
# 2.3 Какой средний чек выше - по экзотическим животным или по кошкам?
a1 = 'Экзотическое животное'
a2 = 'Кошка'
checker(a1, a2)

Между выборками нет статистически значимой разницы


In [8]:
# 2.4 Существуют ли сегменты (разбиение по породе), в рамках которых средний чек по кошкам больше, чем по собакам?

# Ищем сочетания, где средний чек по кошкам больше, чем средний чек по собакам
# Сделаем df, в котором есть все взаимные сочетания по породам для кошек и собак и циклом проверим эти сочетания

df_gr_cat = df_gr[df_gr['animal']=='Кошка'] # Берём группировку только по кошкам
df_gr_dog = df_gr[df_gr['animal']=='Собака'] # Берём группировку только по собакам
df_gr_cat['key'] = 0 # Костыль для кроссджоина
df_gr_dog['key'] = 0 # Костыль для кроссджоина

df_gr_dog_cat = df_gr_dog.merge(df_gr_cat, on='key', how='outer') # Кроссджоин

df_gr_dog_cat['flag'] = np.where(df_gr_dog_cat['avg_check_y']>df_gr_dog_cat['avg_check_x'], 1, 0) # Потенциальные кандидаты
df_gr_dog_cat = df_gr_dog_cat = df_gr_dog_cat[df_gr_dog_cat['flag']==1] # Оставляем только потенциальных кандидатов

df_gr_dog_cat.reset_index(inplace = True) # делаем индекс последовательным
df_gr_dog_cat = df_gr_dog_cat.drop(columns = ['index', 'key', 'flag', 'cnt_orders_y', 
         'cnt_orders_x', 'avg_check_x', 'avg_check_y']) # Удаляем лишние колонки, опционально для красоты если принтить

for i, row in df_gr_dog_cat.iterrows():
    print(i, '===============================================================================')
    print('Проверка комбинации:', row['animal_y'], row['species_y'], 'vs', row['animal_x'], row['species_x'])
    a_s1 = [row['animal_y'], row['species_y']]
    a_s2 = [row['animal_x'], row['species_x']]
    checker_a_s(a_s1, a_s2)
    print()

0 ===============================================================================
Проверка комбинации: Кошка Домашняя породистая vs Собака Домашняя обыкновенная
Между выборками нет статистически значимой разницы

1 ===============================================================================
Проверка комбинации: Кошка Домашняя обыкновенная vs Собака Домашняя породистая
Между выборками есть статистически значимая разница
Средний чек больше по животному: Кошка Домашняя обыкновенная

2 ===============================================================================
Проверка комбинации: Кошка Домашняя породистая vs Собака Домашняя породистая
Между выборками есть статистически значимая разница
Средний чек больше по животному: Кошка Домашняя породистая

3 ===============================================================================
Проверка комбинации: Кошка Уличная vs Собака Домашняя породистая
Между выборками нет статистически значимой разницы

4 ========================================

C:\Users\MrEdward\AppData\Local\Temp\ipykernel_22452\1318596696.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gr_cat['key'] = 0 # Костыль для кроссджоина
C:\Users\MrEdward\AppData\Local\Temp\ipykernel_22452\1318596696.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gr_dog['key'] = 0 # Костыль для кроссджоина


In [9]:
# 2.5 Какой средний чек выше - по экзотическим животным или по домашним породистым кошкам?
a_s1 = ['Экзотическое животное', 'Экзотическое животное']
a_s2 = ['Кошка', 'Домашняя породистая']
checker_a_s(a_s1, a_s2)


Между выборками нет статистически значимой разницы
